In [63]:
#import the packages needed for this question
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [64]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'html.parser')
my_table = soup.find('tbody')


In [65]:
#collecting all the data in the form of table
table_data = []

for row in my_table.findAll('tr'):
    row_data = []
    
    for cell in row.findAll('td'):
        row_data.append(cell.text)
    
    if(len(row_data) > 0):
        data_item = {
            "Postalcode": row_data[0],
            "Borough": row_data[1],
            "Neighbourhood": row_data[2],
        }
    table_data.append(data_item)

In [66]:
# converting table_data into pandas dataframe
df = pd.DataFrame(table_data)
df.head()

,Postalcode,Borough,Neighbourhood
0,M9Z\n,Not assigned\n,\n
1,M1A\n,Not assigned\n,\n
2,M2A\n,Not assigned\n,\n
3,M3A\n,North York\n,Parkwoods\n
4,M4A\n,North York\n,Victoria Village\n


In [67]:
type(df)

pandas.core.frame.DataFrame

In [68]:
df['Neighbourhood'] = df['Neighbourhood'].str.replace('\n','')
df['Neighbourhood'] = df['Neighbourhood'].str.replace('/',',')
df['Borough'] = df['Borough'].str.replace('\n','')
df['Postalcode'] = df['Postalcode'].str.replace('\n','')

In [69]:
# removing the cells where Borough is Not assigned
df_filtered = df.loc[df['Borough'] != 'Not assigned']
df_filtered.head()

,Postalcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park , Harbourfront"
6,M6A,North York,"Lawrence Manor , Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [70]:
df_toronto = df_filtered.groupby(['Postalcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
df_toronto

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov..."
101,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam..."


In [71]:
df_toronto.shape

(103, 3)

In [72]:
print('The number of rows in my dataframe = ',df_toronto.shape[0])

The number of rows in my dataframe =  103


##### This is end of answer for Question 1

## Question 2


##### For me, when I tried to import geocoder python package, I got "ModuleNotFoundError" Hence, I used geographical coordinates of the neighborhoods using csv file from the server

In [73]:
df_locations = pd.read_csv('https://cocl.us/Geospatial_data')
df_locations.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Combining df_torornto and df_locations dataframes into a new dataframe

In [74]:
df_toronto_loc = pd.merge(left=df_toronto ,right=df_locations, left_on='Postalcode', right_on='Postal Code', how='inner')
df_toronto_loc.head()

,Postalcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [75]:
df_toronto_loc.drop(['Postal Code'],axis=1,inplace=True)

In [76]:
df_toronto_loc

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam...",43.739416,-79.588437


In [77]:
df_toronto_loc.shape

(103, 5)

In [78]:
print('The final dataframe now has %d rows and %d columns' %(df_toronto_loc.shape[0], df_toronto_loc.shape[1]))

The final dataframe now has 103 rows and 5 columns


#### This is the end of solution for Question 2